In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("D:\\git\\KJeq\\A1_2103_all.csv", sep=',', header=0, skip_blank_lines=True)


In [ ]:
df.rename(columns={"MULTI_P1_REAL": "P1000", "MULTI_P2_REAL": "P500"}, inplace=True)
df.rename(columns={"MULTI_T1_REAL": "T1000", "MULTI_T2_REAL": "T500"}, inplace=True)
df.Time = df.Date + " " + df.Time
df.Time = pd.to_datetime(df.Time, infer_datetime_format=True)

df.head()

In [ ]:
plt.scatter(df['Time'],df['P500'])

In [ ]:
def hampel_filter_pandas(input_series, window_size, n_sigmas=3):

    k = 1.4826 # scale factor for Gaussian distribution
    new_series = input_series.copy()

    # helper lambda function 
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    
    rolling_median = input_series.rolling(window=2*window_size, center=True).median()
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = np.abs(input_series - rolling_median)

    indices = list(np.argwhere(diff > (n_sigmas * rolling_mad)).flatten())
    new_series[indices] = rolling_median[indices]
    
    return new_series, indices

rw_series = pd.Series(df['P500'])
res, detected_outliers = hampel_filter_pandas(rw_series, 100)


In [ ]:
dfR


In [ ]:

ts = df.loc[:,['P500_time','P500_signal']]
ts['P500_time'] = pd.to_datetime(ts['P500_time'])
ts.set_index('P500_time', inplace=True)
ts.head()

In [ ]:
tsRm = ts['P500_signal'].resample("60T").mean()
tsRup = ts['P500_signal'].resample("60T").max()
tsRlow = ts['P500_signal'].resample("60T").min()
len(tsRup)



In [ ]:
tsRm[:]


500 m, 1000 m 심도에서의 압력, 온도 측정값을 가져옴.

먼저 500 m 심도에서 측정된 압력(P500_signal)에 대해 분석을 실시함.

In [ ]:
plt.scatter(tsRm.index,tsRm)
plt.scatter(tsRup.index,tsRup)
plt.scatter(tsRlow.index,tsRlow)
#plt.plot(tsRlow)
#plt.plot(tsRup)
plt.show()



데이터의 stationarity 체크를 위한 모듈 정의

In [ ]:
from scipy.signal import lfilter

n = 1
  # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1
yy = lfilter(b,a,tsRm)
plt.plot(tsRm.index, yy, linewidth=2, linestyle="-", c="b")  # smooth by filter

In [ ]:
def check_stationarity(ts):
    dftest = adfuller(ts)
    adf = dftest[0]
    pvalue = dftest[1]
    critical_value = dftest[4]['5%']
    if (pvalue < 0.05) and (adf < critical_value):
        print('The series is stationary')
    else:
        print('The series is NOT stationary')

In [ ]:
check_stationarity(ts)

In [ ]:
ts_diff = ts.diff()
ts_diff.dropna(inplace=True)
check_stationarity(ts_diff)

In [ ]:
plt.plot(ts_diff)
plt.title('Differenced Time Series')
plt.grid()
plt.show()

시계열 데이터를 1회 미분했을 때 stationarity가 확보되며 그래프는 상기와 같음.

다음 사이트의 이상감지기법을 적용해 봄.

https://neptune.ai/blog/anomaly-detection-in-time-series

In [ ]:
import statsmodels.tsa.seasonal

plt.rc('figure',figsize=(12,8))
plt.rc('font',size=15)
result = statsmodels.tsa.seasonal.seasonal_decompose(ts,model='additive')
fig = result.plot()

In [ ]:
ts.head()


In [ ]:
len(ts)


In [ ]:
tsm.head()


In [ ]:
import statsmodels.tsa.seasonal

plt.rc('figure',figsize=(16,12))
plt.rc('font',size=15)
result = statsmodels.tsa.seasonal.seasonal_decompose(tsm, model='additive', period=17280)
fig = result.plot()

In [ ]:
result.resid.sort_values()[:40]

